<a href="https://colab.research.google.com/github/Rito43/Rito43/blob/main/M23CSA021_DLOps_ClassAssignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load USPS dataset
train_dataset = datasets.USPS(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.USPS(root='./data', train=False, download=True, transform=transform)

# Set batch size
batch_size = 64

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Testing loop
model.eval()
test_correct = 0
test_total = 0
predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.numpy())
        true_labels.extend(labels.numpy())

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Compute precision and recall
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Epoch [1/10], Train Loss: 0.5592, Train Accuracy: 82.73%
Epoch [2/10], Train Loss: 0.1460, Train Accuracy: 95.68%
Epoch [3/10], Train Loss: 0.0927, Train Accuracy: 97.17%
Epoch [4/10], Train Loss: 0.0562, Train Accuracy: 98.16%
Epoch [5/10], Train Loss: 0.0421, Train Accuracy: 98.74%
Epoch [6/10], Train Loss: 0.0373, Train Accuracy: 98.88%
Epoch [7/10], Train Loss: 0.0239, Train Accuracy: 99.26%
Epoch [8/10], Train Loss: 0.0183, Train Accuracy: 99.41%
Epoch [9/10], Train Loss: 0.0173, Train Accuracy: 99.48%
Epoch [10/10], Train Loss: 0.0147, Train Accuracy: 99.52%
Test Accuracy: 95.37%
Confusion Matrix:
[[351   0   3   0   1   0   3   1   0   0]
 [  0 258   0   0   3   0   3   0   0   0]
 [  2   0 188   1   2   1   1   1   2   0]
 [  0   0   3 156   1   2   0   1   2   1]
 [  0   2   3   0 187   1   2   1   0   4]
 [  2   0   1   3   0 144   3   0   4   3]
 [  0   0   1   0   4   0 165   0   0   0]
 [  0   0   2   0   5   0   0 140   0   0]
 [  4   0   2   1   1   0   1   0 155   2]
 [

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load USPS dataset
train_dataset = datasets.USPS(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.USPS(root='./data', train=False, download=True, transform=transform)

# Set batch size
batch_size = 64

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# TensorBoard writer
writer = SummaryWriter()

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # Log training loss and accuracy
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Testing loop
model.eval()
test_correct = 0
test_total = 0
predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.numpy())
        true_labels.extend(labels.numpy())

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Log test accuracy
writer.add_scalar('Accuracy/test', test_accuracy, 0)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Compute precision and recall
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Log precision and recall
writer.add_scalar('Precision', precision, 0)
writer.add_scalar('Recall', recall, 0)

# Close the TensorBoard writer
writer.close()


Epoch [1/5], Train Loss: 0.5545, Train Accuracy: 82.09%
Epoch [2/5], Train Loss: 0.1395, Train Accuracy: 95.97%
Epoch [3/5], Train Loss: 0.0834, Train Accuracy: 97.52%
Epoch [4/5], Train Loss: 0.0570, Train Accuracy: 98.23%
Epoch [5/5], Train Loss: 0.0374, Train Accuracy: 98.94%
Test Accuracy: 94.72%
Confusion Matrix:
[[353   0   2   0   2   0   1   0   1   0]
 [  1 256   0   0   4   0   3   0   0   0]
 [  3   0 182   0   3   2   0   1   7   0]
 [  1   0   4 141   0  14   0   0   5   1]
 [  0   2   2   0 188   1   3   1   0   3]
 [  3   0   1   2   0 150   0   0   1   3]
 [  0   1   0   0   4   1 164   0   0   0]
 [  2   0   1   0   6   0   0 137   0   1]
 [  6   0   1   0   0   0   0   0 157   2]
 [  0   0   0   0   2   0   0   0   2 173]]
Precision: 0.9464
Recall: 0.9419
